## Quiz 05 Answers
1. False
2. False
3. True
4. False
5. [0.2, 0.8]
6. [0.0, 1.0]

In [ ]:
# Objective 1: Implement teacher forcing in encoder-decoder

import torch
from torch import nn

torch.manual_seed(605)
batch_size = 4
input_length = 16
output_length = 8
hidden_size = 32
vocab_size = 64

encoder = nn.LSTM(hidden_size, hidden_size, 1) # input size, hidden size, num layers
decoder = nn.LSTM(2 * hidden_size, hidden_size, 1)
embedding = nn.Embedding(vocab_size, hidden_size)
output_layer = nn.Linear(hidden_size, vocab_size)

source_ids = (torch.rand(input_length, batch_size) * vocab_size).long()
target_ids = (torch.rand(output_length, batch_size) * vocab_size).long()

input_emb = embedding(source_ids)
output_emb = torch.cat([torch.zeros(1, batch_size, hidden_size), embedding(target_ids[1:, :])], 0) # y_{t-1}, i.e. teacher forcing

encoder_output, (hn, cn) = encoder(input_emb) # hn.size() = [1, batch_size, hidden_size]
decoder_input = torch.cat([output_emb, hn.repeat(output_length, 1, 1)], 2)
decoder_output, _ = decoder(decoder_input)
logits = output_layer(decoder_output)

cel = nn.CrossEntropyLoss()
loss = cel(logits.reshape(output_length * batch_size, vocab_size), target_ids.reshape(output_length * batch_size))
assert (loss - 4.1539).abs() < 0.01

In [ ]:
# Objective 2: Implement a simple (Luong) attention mechanism in encoder-decoder
attention_layer = nn.Linear(hidden_size, hidden_size)
attention_logits = attention_layer(decoder_output.transpose(0, 1)) @ encoder_output.transpose(0, 1).transpose(1, 2) # tWs = (W @ t)^t @ s
softmax = nn.Softmax(2)
attention_weights = softmax(attention_logits)

torch.Size([16, 4, 32])
torch.Size([4, 8, 16])
